In [1]:
import os
import gc
import pickle
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from DebertaGraphLib import CustomExtrDebertaV2ForMultipleChoice, Head, GNN, GraphProcessor
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch, Data
from tqdm.auto import tqdm, trange
from sklearn.metrics import accuracy_score
from transformers import get_cosine_schedule_with_warmup, AutoModel, AutoTokenizer, AutoModelForMultipleChoice
import wandb
pl.seed_everything(56)

Seed set to 56


56

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
class CFG:
    class data:
        train_path = './train_rand_split.jsonl'
        val_path = './dev_rand_split.jsonl'
        test_path = './test_rand_split_no_answers.jsonl'
        tokenizer = 'microsoft/deberta-v3-large'
        graph_path = './graph_data.pkl'
        n_hop = 1
        max_edges = 800
        num_workers = 8
        nfolds = 5
        batch_size = 8 
        use_prefix = False
        max_length = 80 #128
        embeds_path = None#'mean_embeds.npy'
        seed = 56
    class model:
        model = 'microsoft/deberta-v3-large'
        optim=torch.optim.AdamW
        num_labels = 5
        scheduler= 'cosine'
        warnap_steps = 0.0 #0.25
        label_smoothing = 0.0
        pool = 'mean'
        max_epoches = 5
        cls_drop_type = None
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        head_hidden = [1024,1]
        head_act = nn.GELU()
        cls_drop = 0.1
        lr = 3e-6
        lr_gnn = 1.6e-4
        lr_head = 1.6e-4
        turn_off_drop = True
        num_cycles = 0.5
        eps = 1e-7
        weight_decay = 0.0
        gnn_weight_decay = 0.0
        head_weight_decay = 0.0
        betas = (0.9, 0.999)
        class gnn:
            pool_type = 'cls'
            edge_dim = 17
            blocks = [
                {'block_type':'gatv2','act':nn.GELU(),'block_norm':None,'drop_p':0.0,'input_dim':1024,'out_dim':128,'heads':4},
                {'block_type':'gatv2','act':nn.Identity(),'block_norm':None,'drop_p':0.0,'input_dim':512,'out_dim':256,'heads':4}
            ]
    seed = 56
    fold_number = 0

In [4]:
def make_df(path,is_test=False):
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    df['input_concept'] = data['question'].apply(lambda x: x['question_concept'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = 0
    return df

In [5]:
class PLDataset(Dataset):
    def __init__(self, df, tokenizer, gnn_proc, concept_maper):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.tokenizer = tokenizer
        self.gnn_proc = gnn_proc
        self.concept_maper = concept_maper
        self.curent_concepts = set(concept_maper.keys())
        
    def __len__(self):
        return len(self.data)
    
    def get_graph_data(self, input_concept, target_concepts):
        data = []
        input_concept = input_concept.replace(' ','_')
        for target_concept in target_concepts:
            target_concept = target_concept.replace(' ','_')
            if target_concept not in self.curent_concepts:
                target_concept = input_concept
            data.append(Data(**self.gnn_proc.process(self.concept_maper[input_concept],self.concept_maper[target_concept])))
        return data
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        negs = [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]
        if self.cfg.use_prefix:
            prompts = [[f"Question: {row['question']}",f"Answer: {i}"] for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]]
        else:
            prompts = [[row['question'],i] for i in [row['choice_A'],row['choice_B'],row['choice_C'],row['choice_D'],row['choice_E']]]
        
        
        encodes = [
            self.tokenizer.encode_plus(question,answer,
                                       max_length=self.cfg.max_length,
                                       truncation=False,
                                       padding='max_length',
                                       return_tensors='pt')
            for question,answer in prompts
        ]
        
        return {
            'input_ids':torch.stack([x.input_ids.squeeze(0) for x in encodes]),
            'attention_mask':torch.stack([x.attention_mask.squeeze(0) for x in encodes]),
            'token_type_ids':torch.stack([x.token_type_ids.squeeze(0) for x in encodes]),
            'graph_encodes': self.get_graph_data(row['input_concept'],negs),
            'labels': row['label']
        }

In [6]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
        
    def prepare_data(self):
        if self.is_prepared:
            return None
        self.train_df = make_df(self.cfg.train_path)
        self.val_df = make_df(self.cfg.val_path)
        self.test_df = make_df(self.cfg.test_path,is_test=True)
        self.tokenizer = AutoTokenizer.from_pretrained(self.cfg.tokenizer)
        with open(self.cfg.graph_path,'rb') as f:
            self.graph_encodes = pickle.load(f)
        if self.cfg.embeds_path is not None:
             self.graph_encodes['embeds'] = np.load(self.cfg.embeds_path)
        self.graph_processor = GraphProcessor(self.graph_encodes['graph'], self.graph_encodes['embeds'],n_hop=self.cfg.n_hop,max_edges=self.cfg.max_edges)
        self.concept_maper = self.graph_encodes['concepts_maper']
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup:
            return None
        self.train_dataset = PLDataset(self.train_df,self.tokenizer,self.graph_processor,self.concept_maper)
        self.val_dataset = PLDataset(self.val_df,self.tokenizer,self.graph_processor,self.concept_maper)
        self.predict_dataset = PLDataset(self.test_df,self.tokenizer,self.graph_processor,self.concept_maper)
        self.is_setup = True
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         collate_fn=collate_fn,
                         pin_memory=True,
                         shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset,
                          batch_size=self.cfg.batch_size,
                          num_workers=self.cfg.num_workers,
                          collate_fn=collate_fn,
                          pin_memory=True,
                          shuffle=False)    

def compute_idxes(batch):
    idxes = []
    for i in range(len(batch)):
        if batch[i] != batch[i-1]:
            idxes.append(i)
    return torch.tensor(idxes)    
    
def collate_fn(batches):
    new_batch = {}
    for k in batches[0].keys():
        if k == 'graph_encodes':
            values = []
            for x in batches:
                values += x[k]
            new_batch[k] = Batch.from_data_list(values)
        elif k == 'labels':
            new_batch[k] = torch.from_numpy(np.array([x[k] for x in batches]))
        else:
            new_batch[k] = torch.stack([x[k] for x in batches])
            
    new_batch['cls_graph_dims'] = compute_idxes(new_batch['graph_encodes'].batch)
    return new_batch

In [7]:
class AverageMeter():
    def __init__(self):
        self.preds = []
        self.labels = []
    
    def update(self,y_t,y_p):
        self.labels += y_t
        self.preds += y_p
        
    def clean(self):
        self.preds = []
        self.labels = []

    def calc_metrics(self):
        metrics = {}
        metrics['accuracy'] = accuracy_score(self.labels, self.preds)
        
        return metrics

In [8]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.text_encoder = CustomExtrDebertaV2ForMultipleChoice(self.cfg)
        self.criterion = nn.CrossEntropyLoss(label_smoothing=self.cfg.label_smoothing)
        self.graph_encoder = GNN(self.cfg.gnn)
        self.avg_meter = AverageMeter()
        self.head = Head(self.cfg)
        
    def forward(self, batch):
        gnn_ids = self.graph_encoder(
            x=batch['graph_encodes'].x,
            edge_index=batch['graph_encodes'].edge_index,
            edge_attr=batch['graph_encodes'].edge_attr,
            batch=batch['graph_encodes'].batch,
            cls_idxes=batch['cls_graph_dims'],
        ) * 0.0
        
        text_feats = self.text_encoder(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            gnn_ids=gnn_ids,
        )

        logits = self.head(text_feats)
        return logits.view(-1, self.cfg.num_labels)

    def training_step(self, batch, i):
        out = self(batch)
        loss = self.criterion(out,batch['labels'])
        self.log('train_loss', loss.item())
        return loss
            
    def validation_step(self, batch, i):
        out = self(batch)
        loss = self.criterion(out,batch['labels'])
        self.log('val_loss',loss.item())
        preds = out.argmax(dim=-1).tolist()
        self.avg_meter.update(batch['labels'].tolist(),preds)
    
    def predict_step(self, batch, i):
        logits = self(batch)
        return logits.argmax(dim=-1).tolist()
                
    def on_validation_epoch_end(self):
        metrics = self.avg_meter.calc_metrics()
        self.log_dict(metrics)
        self.avg_meter.clean()
            
    def configure_optimizers(self):
        optimizer_parameters = [
            {'params': [p for n, p in self.text_encoder.model.named_parameters() if not any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': [p for n, p in self.text_encoder.model.named_parameters() if any(nd in n for nd in self.cfg.no_decay)],
             'lr': self.cfg.lr, 'weight_decay': 0.0},
            {'params': [p for n, p in self.graph_encoder.named_parameters()],
             'lr': self.cfg.lr_gnn, 'weight_decay': self.cfg.gnn_weight_decay},
            {'params': [p for n, p in self.head.named_parameters()],
             'lr': self.cfg.lr_head, 'weight_decay': self.cfg.head_weight_decay},
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warnap_steps,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warnap_steps)
        else:
            scheduler = torch.optim.lr_scheduler.ConstantLR(optim)
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [9]:
dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
CFG.model.num_training_steps = (len(dm.train_dataloader())) * CFG.model.max_epoches

In [11]:
model = PLModule()

In [12]:
wandb.login(key="31520b01739d418e5d77a11fd8a79a70b189b8bc")
os.environ['WANDB_API_KEY'] = "31520b01739d418e5d77a11fd8a79a70b189b8bc"
wandb.init(project='AIIJC_task1',name='debertav3_large')

wandb: Currently logged in as: andrewkhl (andlh). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
checkpoint_cb = pl.callbacks.ModelCheckpoint(
    dirpath='./outputs/',
    filename='model_{epoch:02d}-{accuracy:.4f}',
    monitor='accuracy',
    mode='max',
    save_last=True
)

trainer = pl.Trainer(
    accelerator="gpu",
    precision=32,
    callbacks = [lr_monitor,checkpoint_cb],
    logger = pl.loggers.WandbLogger(save_code=True),
    log_every_n_steps=1,
    min_epochs=1,
    devices=1,
    check_val_every_n_epoch=1,
    max_epochs=CFG.model.max_epoches
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, datamodule=dm)

You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:652: Checkpoint directory /notebooks/outputs exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                                 | Params | Mode 
-------------------------------------------------------------------------------
0 | text_encoder  |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.c

Training: |          | 0/? [00:00<?, ?it/s]

/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.c

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 5. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: Us

Validation: |          | 0/? [00:00<?, ?it/s]

/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  'edge_attr': torch.tensor(edge_attrs),
/notebooks/DebertaGraphLib/graph_data_utils.py:37: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.c